In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import glob

# 1. Find all CSVs in our raw data folder
files = glob.glob("../data/raw/*.csv")

plt.figure(figsize=(12, 6))

for file in files:
    # Get the ticker name from the filename
    ticker = file.split('/')[-1].replace('.csv', '')
    
    # Load data
    df = pd.read_csv(file, index_col='Date', parse_dates=True)
    
    # Calculate Percentage Change from the start of the month
    # Formula: ((Current Price / Starting Price) - 1) * 100
    initial_price = df['Close'].iloc[0]
    pct_growth = ((df['Close'] / initial_price) - 1) * 100
    
    # Plot the growth
    plt.plot(pct_growth, label=f'{ticker}')

# 3. Formatting the "Robust" Chart
plt.title('Percentage Growth Comparison (Last 1 Month)')
plt.ylabel('Growth %')
plt.xlabel('Date')
plt.axhline(0, color='black', linewidth=1, linestyle='--') # Baseline
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()